In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
# импортируем класс DecisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier
# импортируем функцию cross_val_score() 
# и классы RepeatedStratifiedKFold,
# ParameterGrid, GridSearchCV
from sklearn.model_selection import (cross_val_score,
                                     RepeatedStratifiedKFold,
                                     ParameterGrid,
                                     GridSearchCV)

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Bankloan.csv', 
                   encoding='cp1251', 
                   sep=';', decimal=',')

# создаем массив меток и массив признаков
y = data.pop('default')
X = pd.get_dummies(data)

# создаем экземляр класса DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=152)

In [3]:
# задаем сетку гиперпараметров
param_grid = {'max_depth': [4, 8, 12], 'max_features': [3, 6]}

# запускаем вложенную перекрестную проверку
scores = cross_val_score(GridSearchCV(tree, param_grid, cv=5),
                         X, y, cv=5)
print("Значения правильности: ", scores)
print("Среднее значение правильности: ", scores.mean())

Значения правильности:  [0.71333333 0.70333333 0.68       0.72333333 0.69666667]
Среднее значение правильности:  0.7033333333333334


In [4]:
# пишем функцию, выполняющую вложенную
# перекрестную проверку
def nested_cv(X, y, inner_cv, outer_cv, Classifier, parameter_grid):
    """
    Автор: Andreas Mueller <t3kcit@gmail.com>
    
    Выполняет вложенную перекрестную проверку.
    
    Параметры
    ----------
    X: numpy.ndarray
        Массив признаков.
    y: numpy.ndarray
        Массив меток.
    inner_cv: int
        Количество итераций внутренней
        перекрестной проверки.
    outer_cv: int
        Количество итераций внешней
        перекрестной проверки.
    Classifier: instance of class sklearn
        Классификатор.
    parameter_grid: dict of str
        Сетка гиперпараметров.
    
    Возвращает
    ----------
    results: pandas.DataFrame
        Датафрейм с наилучшими гиперпараметрами
        и наилучшим значением метрики.
    """
    # собираем наилучшие значения правильности
    # и наилучшие гиперпараметры
    outer_scores_lst = []
    best_params_lst = []
    # для каждого разбиения данных во внешней перекрестной проверке
    # (метод split возвращает индексы)
    for outer_cnt, (training_samples, test_samples) in enumerate(
        outer_cv.split(X, y), 1):
        # находим наилучшие значения гиперпараметров 
        # с помощью внутренней перекрестной проверки
        best_params = {}
        best_score = -np.inf
        # итерируем по гиперпараметрам
        for parameters in parameter_grid:
            # собираем значения правильности 
            # по всем внутренним разбиениям
            cv_scores = []
            # итерируем по разбиениям внутренней перекрестной проверки 
            for inner_cnt, (inner_train, inner_test) in enumerate(
                inner_cv.split(X[training_samples], y[training_samples]), 1):
                # строим классификатор с данными значениями гиперпараметров
                # на внутренней обучающей выборке
                clf = Classifier(**parameters)
                clf.fit(X[inner_train], y[inner_train])
                # оцениваем качество на внутренней тестовой выборке
                score = clf.score(X[inner_test], y[inner_test])
                cv_scores.append(score)
            # вычисляем среднее значение правильности 
            # по внутренним тестовым выборкам
            mean_score = np.mean(cv_scores)
            if mean_score > best_score:
                # если лучше, чем предыдущие, записываем 
                # значения гиперпараметров как лучшие
                best_score = mean_score
                best_params = parameters
        # строим классификатор с наилучшими значениями 
        # гиперпараметров на внешней обучающей выборке
        clf = Classifier(**best_params)
        clf.fit(X[training_samples], y[training_samples])
        # оцениваем качество на внешней тестовой выборке
        out_score = clf.score(X[test_samples], y[test_samples])
        # добавляем лучшие значения метрики 
        # и лучшие гиперпараметры в списки
        outer_scores_lst.append(out_score)
        best_params_lst.append(best_params)
    # создаем словарь с наилучшими гиперпараметрами
    # и наилучшими значениями метрики
    results = pd.DataFrame(best_params_lst)
    results['score'] = outer_scores_lst
    return results

In [5]:
# задаем сетку гиперпараметров
param_grid = {'max_features': [1, 2, 3], 
              'max_depth': [2, 4, 6]}

# применяем нашу функцию
scores = nested_cv(X.values, 
                   y.values, 
                   RepeatedStratifiedKFold(n_repeats=5,
                                           n_splits=5, 
                                           random_state=42), 
                   RepeatedStratifiedKFold(n_repeats=5,
                                           n_splits=5,
                                           random_state=42),
                   DecisionTreeClassifier, 
                   ParameterGrid(param_grid))
scores

,max_depth,max_features,score
0,6,3,0.693333
1,4,3,0.713333
2,6,3,0.726667
3,4,3,0.713333
4,6,3,0.673333
5,6,3,0.703333
6,4,3,0.733333
7,6,3,0.656667
8,6,3,0.656667
9,6,3,0.710000
